In [27]:
import datetime
import psycopg2 # Postgresql
import pandas as pd # Ez table management
from dotenv import load_dotenv # Load .env strings
import os # console commands
import boto3 # AWS querier

In [6]:
# Note: to load .env video-annotation-tool must be one directory up
load_dotenv(dotenv_path="../.env")
S3_BUCKET = os.getenv('AWS_S3_BUCKET_NAME')
SRC_IMG_FOLDER = os.getenv('AWS_S3_BUCKET_ANNOTATIONS_FOLDER')
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
A_KEY = os.getenv("AWS_ACCESS_KEY_ID")
A_SEC = os.getenv("AWS_SECRET_ACCESS_KEY")

In [7]:
# Connect to database using .env variables
def queryDB(query, params=None):
    conn = psycopg2.connect(database = DB_NAME,
                        user = DB_USER,
                        password = DB_PASSWORD,
                        host = DB_HOST,
                        port = "5432")
    # Use pandas to read queries into a dataframe
    result = pd.read_sql_query(query, conn, params=params)
    conn.close() # Close postgresql
    return result

# Remove old csv of s3Images

In [8]:
# Remove old s3Images
os.remove('s3Images.csv')

FileNotFoundError: [Errno 2] No such file or directory: 's3Images.csv'

# Create csv of image names in lubo's s3 bucket

In [9]:
# Get all the image filename
with open('s3Images.csv','a') as fd:
    # Connect to AWS
    session = boto3.Session(aws_access_key_id = A_KEY, aws_secret_access_key = A_SEC)
    # Access simple storage and s3 bucket
    s3 = session.resource('s3')
    bucket = s3.Bucket("lubomirstanchev")
    # Iterate through bucket folder test
    for index, obj in enumerate(bucket.objects.filter(Delimiter='/',Prefix='test/')):
        if index >= 1:
            # Make csv of images in s3 bucket
            fd.write(obj.key + '\n')

# Get all image names in database

In [10]:
database = queryDB('''
    Select
        annotations.*, videos.filename
    FROM
        annotations
    LEFT JOIN
        videos
    ON
        videos.id=videoid
''')

# Read in s3 image names into memory

In [11]:
s3Images = pd.read_csv("s3Images.csv", header=None)

# Number of images in database

In [12]:
database.shape[0]

2340907

# Remove extra path info in s3 images names

In [13]:
s3Images[0] = s3Images[0].str.split('/', expand=True)[1]
s3Images = s3Images[0]

# Remove images that exist in the s3 bucket

In [14]:
database = database[~(database.image.isin(s3Images) & database.imagewithbox.isin(s3Images))]

# Number of images that are missing from the s3 bucket

In [15]:
print(f"There are {database.shape[0]} missing images")

There are 81450 missing images


# Explore info about missing images

In [16]:
database.userid.value_counts()

32     80330
13       944
15       106
171       20
569       17
4         15
7          8
12         7
6          2
5          1
Name: userid, dtype: int64

In [17]:
database.conceptid.value_counts()

347     23314
236      8654
2136     7369
2137     6287
506      5235
        ...  
2187        1
359         1
137         1
889         1
1601        1
Name: conceptid, Length: 61, dtype: int64

In [21]:
database.dateannotated.value_counts()

2019-08-17    15315
2019-11-30    12261
2019-12-01    10119
2019-08-16     8351
2019-08-20     7900
2019-08-30     7450
2019-08-18     6859
2019-08-15     5617
2019-08-21     2349
2019-08-19     2096
2019-08-28     1176
2019-08-31      720
2019-11-21      398
2019-12-06      140
2019-12-11      114
2019-09-16      103
2019-09-03      100
2019-12-10       59
2019-11-20       49
2019-11-22       43
2019-11-28       40
2019-09-06       20
2020-01-08       20
2019-11-13       17
2019-12-18       17
2019-10-23       13
2019-12-05       11
2019-12-23       10
2019-12-28        9
2019-10-31        9
2019-12-17        9
2019-11-27        8
2019-09-21        5
2020-01-10        5
2019-10-24        4
2019-07-30        3
2019-07-31        3
2019-01-27        2
2019-10-18        2
2019-12-03        2
2019-09-12        2
2019-08-23        2
2020-01-16        2
2019-12-02        2
2019-11-07        2
2019-09-20        1
2020-01-15        1
2019-09-18        1
2019-09-13        1
2019-02-07        1


In [22]:
database.columns

Index(['id', 'videoid', 'userid', 'conceptid', 'timeinvideo', 'x1', 'y1', 'x2',
       'y2', 'videowidth', 'videoheight', 'dateannotated', 'image',
       'imagewithbox', 'comment', 'unsure', 'originalid', 'framenum', 'speed',
       'verifieddate', 'verifiedby', 'priority', 'oldconceptid', 'oldx1',
       'oldy1', 'oldx2', 'oldy2', 'tracking_flag', 'filename'],
      dtype='object')

# Save missing images to csv

In [54]:
# This is used for generation of missing images
database.to_csv(f"missingImages {datetime.datetime.now().replace(microsecond=0)}", index=False)